In [2]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [4]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

In [45]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
best_path_skim_dir = params['best_path_skim_dir']

perf_measure_columns = params['final_columns']

In [40]:
def post_processing_station_paris(stns_df):
    """ add time of day, clean station names, 
        
        selecting the row (of the set of duplicates) with the lowest number of 
        transfers by time period and station pair. 
        
        Station names – in GTFS, each station is represented by at least 2 platforms (e.g., northbound and southbound). 
        So we should group platforms together that belong to the same station. Please note that these GTFS files 
        include the HSR stations, including ones in the Los Angeles area.
        
        Time period – for GTFS processing, I’ve developed 6 time periods, with overnight being 
        encompassing 1900-2600 (late night services that run past midnight are coded as 24:00:00 in GTFS) and 
        0000-0300. These two periods will need to be combined into a single ‘overnight’ time period.
        
        Duplicate rows – After doing the above two tests, you’ll have a number of duplicate rows for 
        each station pair and time period. This is because there are paths from many time points: 
        sometimes a 1-transfer movement is faster than a no-transfer movement, sometimes it’s vice-versa. 
        Because this is a connectivity metric, I would suggest selecting the row (of the set of duplicates) with the lowest number of transfers by time period and station pair. 
        High-Speed Rail – The GTFS files include HSR, so you might want to/need to filter out those station pairs. I am not sure if the BC team wants to include those; I believe the metric is for “regional rail/BART” services.
        
        Summarization – after the above three steps, you’ll have to develop the metric. 
        I think they want the number of station pairs by 0/1/2/3+ transfers. 

    """
    
    
    tod_dict = {'0300-0600' : 'EA', 
                '0600-1000' : 'AM',
                '1000-1500' : 'MD',
                '1500-1900' : 'PM',
                '1900-2600' : 'EV'}
    
    
    columns = ['stop_name.x', 'stop_name.y']
    strngs_replace = [' - Northbound', ' - Southbound', ' - Westbound', ' - Eastbound']

    for col in columns:
        for strng in strngs_replace:
            stns[col] = stns[col].str.replace(strng,'')
    
    stns['Period'] = stns['time_period'].map(tod_dict)
    
    stn_pairs = stns.groupby(['stop_name.x', 'stop_name.y', 'Period'])['transfers'].min().reset_index()
    
    
    #TO DO Ask BC team about removing HSR stations
    stn_pairs.loc[stn_pairs['transfers']>=2, 'transfers'] = '2+'
    stn_pairs['Value'] = 1
    
    trnsfers = stn_pairs.groupby(['Period', 'transfers'])['Value'].sum().reset_index()
    
    return trnsfers

In [63]:
def create_transfers_performance_measure(stn_pairs, concept_id):
    """populates all the relevant columns for final performance measure
    """
    
    stn_pairs = stn_pairs.rename(columns={'transfers' : 'Zone_ID'})
    stn_pairs['Concept_ID'] = concept_id
    stn_pairs['Metric_ID'] = 'A1.11'
    stn_pairs['Metric_name'] = 'Network integration (stations)'
    stn_pairs['Submetric'] = ''
    stn_pairs['Description'] =  'Connectivity at different number of transfers. The Zone ID refers to number of transfers'
    stn_pairs['Population'] = 'Whole Population'
    stn_pairs['Geography'] = 'Regional'
    stn_pairs['Origin_zone'] = ''
    stn_pairs['Dest_zone'] = ''
    stn_pairs['Purpose'] = ''
    stn_pairs['Units'] = 'Number of station pairs'
    stn_pairs['Total_Increment'] = ''
    
    return stn_pairs

## Baseline

In [64]:
concept = 'baseline'
stns = pd.read_csv(_join(params['common_dir'], r"StationToStaion/baseline.csv"))

stn_pairs = post_processing_station_paris(stns)
final_df = create_transfers_performance_measure(stn_pairs, 'baseline')
final_df = final_df[perf_measure_columns]

final_df

,Concept_ID,Metric_ID,Metric_name,Submetric,Description,Population,Period,Geography,Zone_ID,Origin_zone,Dest_zone,Purpose,Value,Units,Total_Increment
0,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,AM,Regional,0,,,,3364,Number of station pairs,
1,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,AM,Regional,1,,,,2209,Number of station pairs,
2,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,AM,Regional,2+,,,,458,Number of station pairs,
3,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EA,Regional,0,,,,3033,Number of station pairs,
4,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EA,Regional,1,,,,890,Number of station pairs,
5,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EA,Regional,2+,,,,154,Number of station pairs,
6,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EV,Regional,0,,,,3430,Number of station pairs,
7,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EV,Regional,1,,,,2603,Number of station pairs,
8,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EV,Regional,2+,,,,732,Number of station pairs,
9,baseline,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,MD,Regional,0,,,,3428,Number of station pairs,


## for R36 Concept

In [65]:
concept = 'R36'
stns = pd.read_csv(_join(params['common_dir'], r"StationToStaion/R36.csv"))

stn_pairs = post_processing_station_paris(stns)
final_df = create_transfers_performance_measure(stn_pairs, 'R36')
final_df = final_df[perf_measure_columns]

final_df

,Concept_ID,Metric_ID,Metric_name,Submetric,Description,Population,Period,Geography,Zone_ID,Origin_zone,Dest_zone,Purpose,Value,Units,Total_Increment
0,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,AM,Regional,0,,,,3795,Number of station pairs,
1,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,AM,Regional,1,,,,3005,Number of station pairs,
2,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,AM,Regional,2+,,,,795,Number of station pairs,
3,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EA,Regional,0,,,,3430,Number of station pairs,
4,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EA,Regional,1,,,,1275,Number of station pairs,
5,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EA,Regional,2+,,,,141,Number of station pairs,
6,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EV,Regional,0,,,,3910,Number of station pairs,
7,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EV,Regional,1,,,,3383,Number of station pairs,
8,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,EV,Regional,2+,,,,1735,Number of station pairs,
9,R36,A1.11,Network integration (stations),,Connectivity at different number of transfers....,Whole Population,MD,Regional,0,,,,3880,Number of station pairs,


## for R37 Concept

In [62]:
concept = 'R37'
stns = pd.read_csv(_join(params['common_dir'], r"StationToStaion/R37.csv"))

stn_pairs = post_processing_station_paris(stns)
final_df = create_transfers_performance_measure(stn_pairs, 'R37')
final_df = final_df[perf_measure_columns]

final_df

,Concept_ID,Metric_ID,Metric_name,Submetric,Description,Population,Period,Geography,Zone_ID,Origin_zone,Dest_zone,Purpose,Value,Units,Total_Increment
0,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,AM,Regional,0,,,,3755,Number of station pairs,
1,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,AM,Regional,1,,,,4710,Number of station pairs,
2,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,AM,Regional,2+,,,,2614,Number of station pairs,
3,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,EA,Regional,0,,,,3457,Number of station pairs,
4,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,EA,Regional,1,,,,2573,Number of station pairs,
5,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,EA,Regional,2+,,,,620,Number of station pairs,
6,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,EV,Regional,0,,,,3886,Number of station pairs,
7,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,EV,Regional,1,,,,5459,Number of station pairs,
8,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,EV,Regional,2+,,,,3385,Number of station pairs,
9,R37,A1.11,Network integration (stations),,Connectivity at different number of transfers,Whole Population,MD,Regional,0,,,,3884,Number of station pairs,
